Imports & Basic Setup

In [22]:
############################### IMPORTS ###############################

#imports
import random
import pandas as pd
import csv
from collections import defaultdict
from fractions import Fraction

#empty cart
cart = {}

############################### CSV HELPER FUNCTIONS ###############################
def read_recipe_csv(filename):
    recipes = {}
    with open(filename) as f:
        lines = f.readlines()[1:]
        for line in lines:
            recipe, cuisine, servings, time_to_cook, difficulty, calories, ingredient, quantity, unit = line.strip().split(',')
            if recipe not in recipes:
                recipes[recipe] = {
                    'cuisine': cuisine,
                    'servings': int(servings),
                    'time_to_cook': int(time_to_cook),
                    'difficulty': difficulty,
                    'calories': int(calories),
                    'ingredients': {}
                }
            recipes[recipe]['ingredients'][ingredient] = {
                'quantity': float(Fraction(quantity)),
                'unit': unit
            }
    return recipes

def read_ingredients_csv(filename):
    grocery_ingredients = {}
    with open(filename) as f:
        lines = f.readlines()[1:]
        for line in lines:
            ingredient, price, category, unit = line.strip().split(',')
            grocery_ingredients[ingredient] = {
                'price': float(price),
                'category': category,
                'unit': unit
            }
    return grocery_ingredients

def store_checkout_info(cart, address, phone, order_no):
    # Write the cart and customer info to the orders.csv file
    with open('orders.csv', 'a') as csvfile:
        writer = csv.writer(csvfile)
        for item, item_data in cart.items():
            total_price = item_data['quantity'] * grocery_ingredients[item]['price']
            writer.writerow([order_no, phone, address, item, item_data['quantity'], total_price ,item_data['type'],item_data['category']])

def generate_customer_table(input_file: str, output_file: str = "customer_info.csv"):
    # Read data from input CSV file
    with open(input_file, "r") as csvfile:
        csv_reader = csv.reader(csvfile)
        next(csv_reader)  # Skip the header row

        # Prepare the header for the output CSV file
        header = ["phone", "address", "total_orders", "total_money_spent"]

        # Extract customer info and calculate total orders and money spent
        customer_info = defaultdict(lambda: [set(), 0.0])  # Initialize unique order ids and amounts to 0
        for row in csv_reader:
            order_no, address, phone, item, total_qty, total_price, type, cat = row
            key = (address, phone)

            # Update the total number of orders and money spent
            customer_info[key][0].add(order_no)
            customer_info[key][1] += float(total_price)

        # Save the aggregated customer info to a CSV file
        with open(output_file, "w", newline="") as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(header)
            for key, value in customer_info.items():
                csv_writer.writerow([*key, len(value[0]), value[1]])

input_file = "orders.csv"
generate_customer_table(input_file)

############################### DATA DICTIONARIES ###############################

# recipes dictionary
recipes = read_recipe_csv('recipes.csv')

# Define the grocery ingredients with their price and unit of measurement
grocery_ingredients = read_ingredients_csv('grocery_ingredients.csv')

# Dictionary of ingredient categories from grocery_ingredients, keys are categories, values are lists of ingredients
grocery_categories = {grocery_ingredients[ing]["category"]:[] for ing in grocery_ingredients}
recipe_cuisines = {recipes[dish]["cuisine"]:[] for dish in recipes}

############################### HELPER FUNCTIONS ################################

#function to calculate price of a recipe
def calculate_price(recipe):
    total_cost = 0
    for ingredient in recipe['ingredients']:
        total_cost += grocery_ingredients[ingredient]['price'] * recipe['ingredients'][ingredient]['quantity']
    return total_cost

#function to check if the input is a number
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

## function to match the input value to key in a dictionary
def match_master(input_str, dict_obj):
    """
    Returns a list of all keys in the dictionary object that match any part of the input string (case-insensitive).
    Only considers tokens that are at least 3 characters long.
    """
    input_tokens = [t.lower() for t in input_str.split() if len(t) >= 3]
    matching_keys = []
    for key in dict_obj.keys():
        key_tokens = [t.lower() for t in key.split() if len(t) >= 3]
        for input_token in input_tokens:
            for key_token in key_tokens:
                if input_token in key_token or key_token in input_token:
                    matching_keys.append(key)
                    break
    return matching_keys


def print_categorized_items(items_dict, category):
    """
    Prints a list of items categorized by a category.

    Parameters:
    items_dict (dict): The dictionary of items to print.
    category (str): The category to print the items by.
    """
    # group items by category
    categorized_items = {}
    for item in items_dict:
        if items_dict[item][category] not in categorized_items:
            categorized_items[items_dict[item][category]] = []
        categorized_items[items_dict[item][category]].append(item)

    # print items by category with 3 items per line with numbers in consecutive order
    item_number = 1
    max_item_width = max([len(item) for category in categorized_items for item in categorized_items[category]])
    for category in categorized_items:
        print("-" * (max_item_width + 4))
        print(f"{category}:")
        print()
        items_printed = 0
        for item in categorized_items[category]:
            item_str = f"{item_number}. {item}"
            padded_item_str = item_str.ljust(max_item_width + 4) # add 4 spaces to account for the item number and the space between the number and the item
            if items_printed % 3 == 0:
                print(padded_item_str, end='')
            else:
                print(padded_item_str, end='')
            item_number += 1
            items_printed += 1
            if items_printed % 3 == 0:
                print()
        if items_printed % 3 != 0:
            print()

#ordinal
def ordinal(num):
    suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
    suffix = suffixes.get(num % 10 if num % 100 not in [11, 12, 13] else 0, 'th')
    return str(num) + suffix


Scenarios

In [23]:
#1 Find & order ingredients by name – eg. Banana


def add_ingredient_to_cart(ingredient):

    

    #print a list of ingredients based on the user input
    print("Here are the ingredients that match your search:")
    matching_ingredients = match_master(ingredient, grocery_ingredients)
    #print header border
    print("--------------------------------------------------")
    #print header
    print("Ingredient".ljust(20) + "Price".ljust(10) + "Category".ljust(10) + "Unit".ljust(10))
    #print header border
    print("--------------------------------------------------")
    #print ingredients
    for ingredient in matching_ingredients:
        print(ingredient.ljust(20) + str(grocery_ingredients[ingredient]['price']).ljust(10) + grocery_ingredients[ingredient]['category'].ljust(10) + grocery_ingredients[ingredient]['unit'].ljust(10))
    print("--------------------------------------------------")
    print()


    for ingredient in matching_ingredients:


        #check if ingredient is in the cart
        if ingredient not in cart:
            #ask the user for the quantity
            print(f"How many {grocery_ingredients[ingredient]['unit']} of {ingredient} would you like to add?")
            quantity = input("Enter a number: ")

            #check if the user input is a number
            if is_number(quantity):
                #add the ingredient to the cart
                cart[ingredient] = {"quantity": float(quantity), "unit": grocery_ingredients[ingredient]["unit"], "type": "XX", "category":grocery_ingredients[ingredient]["category"]}
                print(f"{quantity} {ingredient} has been added to your cart.")
                print()
            else:
                print("Sorry, that is not a valid option. Please try again.")
                chat_loop()

        else:
            #ask the user for the quantity
            print(f"How many {grocery_ingredients[ingredient]['unit']} of {ingredient} would you like to add?")
            quantity = input("Enter a number: ")

            #check if the user input is a number
            if is_number(quantity):
                #add the ingredient to the cart
                cart[ingredient]["quantity"] += float(quantity)
                print(f"{ingredient} has been added to your cart.")
                print()
            else:

                print("Sorry, that is not a valid option. Please try again.")
                chat_loop()


In [24]:
#2 Find & order ingredients by category – eg. Dairy

def ingridient_category(ingredient_category):

    #match of user input to keys in grocery_categories dictionary
    matching_keys = match_master(ingredient_category, grocery_categories)
    
    #print header border
    print("--------------------------------------------------")
    #print header
    print("Ingredient".ljust(20) + "Price".ljust(10) + "Category".ljust(10) + "Unit".ljust(10))
    #print header border
    print("--------------------------------------------------")
    #print ingredients

    #print filtered ingredients from grocery_ingredients dictionary if category in matching_keys
    for ingredient in grocery_ingredients:
        if grocery_ingredients[ingredient]['category'] in matching_keys:
            print(ingredient.ljust(20) + str(grocery_ingredients[ingredient]['price']).ljust(10) + grocery_ingredients[ingredient]['category'].ljust(10) + grocery_ingredients[ingredient]['unit'].ljust(10))
    print("--------------------------------------------------")
    print()

    #ask the user to select an ingredient
    print("Please select an ingredient from the list above.")
    ingredient_input = input("Enter the name of the ingredient: ").lower()

    #check if user iput is valid
    if len(match_master(ingredient_input, grocery_ingredients)) == 0:
        print("Sorry, that is not a valid option. Please try again.")
        chat_loop()

    #match the user input to the ingredient list
    match_input = match_master(ingredient_input, grocery_ingredients)
    

    for ingredient in match_input:

        #check if ingredient is in the cart
        print(f"How many {grocery_ingredients[ingredient]['unit']} of {ingredient} would you like to add?")
        quantity = input("Enter a number: ")

        #check if the user input is a number
        if is_number(quantity):
            #add the ingredient to the cart
            cart[ingredient] = {"quantity": float(quantity), "unit": grocery_ingredients[ingredient]["unit"], "type": "XX", "category":grocery_ingredients[ingredient]["category"]}
            print(f"{quantity} {ingredient} has been added to your cart.")
            print()
        else:

            print("Sorry, that is not a valid option. Please try again.")
            chat_loop()


In [25]:
#3 Find & order ingredients by Recipes – eg. Chicken Soup

def add_recipe_to_cart(recipe_name):

    #print a list of recipes based on the user input
    print("Here are the recipes that match your search:")
    matching_recipes = match_master(recipe_name, recipes)
    #print header border
    print("--------------------------------------------------")
    #print header
    print("Recipe".ljust(20) + "Servings".ljust(10) + "Time".ljust(10) + "Price".ljust(10))
    #print header border
    print("--------------------------------------------------")
    #print recipes
    for recipe in matching_recipes:
        print(recipe.ljust(20) + str(recipes[recipe]['servings']).ljust(10) + str(recipes[recipe]['time_to_cook']).ljust(10) + str(calculate_price(recipes[recipe])).ljust(10))

    #ask the user to select a recipe or press enter to go back
    print()
 

    #match the user input to the recipe list
    
    match_input =[matching_recipes[0]]
    recipe_name = match_input[0]
    recipe = recipes[recipe_name]

    
    #print servings & time
    print("--------------------------------------------------")
    print(f"Recipe: {recipe_name}")
    print(f"Servings: {recipe['servings']}")
    print(f"Time: {recipe['time_to_cook']}")
    print(f"Price: ${calculate_price(recipe)}")
    #print ingredients
    print("--------------------------------------------------")
    print("Ingredients:")
    for ingredient, quantity in recipe["ingredients"].items():
            print(f"{ingredient:<30} {quantity['quantity']:>5} {quantity['unit']:<10}")

    #confirm if the user wants to add the recipe to the cart
    print()
    print(f"Would you like to add {recipe_name} to your cart? (y/n)")
    user_input = input("Enter your choice: ")

    #check if the user input is valid
    if user_input.lower() not in ["y", "n"]:
        print("Sorry, that is not a valid option. Please try again.")
        add_recipe_to_cart(recipe_name)

    #if the user input is "y", add the recipe to the cart
    if user_input.lower() == "y":
        for ingredient, quantity in recipe["ingredients"].items():
            if ingredient not in cart:
                cart[ingredient] = quantity
                cart[ingredient]["type"] = recipe['cuisine']
                cart[ingredient]['category'] = recipe_name
            else:
                cart[ingredient]["quantity"] += quantity["quantity"]

        print(f"{recipe_name} has been added to your cart.")
        chat_loop()

    #if the user input is "n", go back to the main menu
    if user_input.lower() == "n":
        chat_loop()


In [26]:
#4 Find recipes & order based on cuisine – eg. Korean

def recipe_cuisine(cuisine):

    #match the user input to the cuisine list
    match_input = match_master(cuisine, recipe_cuisines)
    

    #print the recipes in the cuisine
    print(f"Here are the recipes in the {cuisine} cuisine:")
    print()

    #print header border
    print("-" * 50)
    #print the recipes name, servings, time, and price
    print(f"{'Recipe':<30}{'Servings':<10}{'Time':<10}{'Price':<10}")
    print("-" * 50)

    #print filtered recipes from recipes dictionary if cuisine in matching_keys
    for recipe in recipes:
        if recipes[recipe]['cuisine'] in match_input:
            print(f"{recipe:<30}{recipes[recipe]['servings']:<10}{recipes[recipe]['time_to_cook']:<10}${calculate_price(recipes[recipe]):<10}")

    
    #ask the user to select a recipe or press enter to go back
    print()
    print("Please select a recipe from the list above or press enter to go back.")



    recipe_input = input("Enter the name of the recipe: ").lower()

    #if the user presses enter, go back to the main menu
    if recipe_input == "":
        chat_loop()

    
    #match the user input to the recipe list
    match_input = match_master(recipe_input, recipes)
    recipe_name = match_input[0]
    recipe = recipes[recipe_name]

    
    #print servings & time
    print(recipe_name)
    print("--------------------------------------------------")
    print(f"Servings: {recipe['servings']}")
    print(f"Time: {recipe['time_to_cook']}")
    print(f"Price: ${calculate_price(recipe)}")
    #print ingredients
    print("--------------------------------------------------")
    print("Ingredients:")
    for ingredient, quantity in recipe["ingredients"].items():
            print(f"{ingredient:<30} {quantity['quantity']:>5} {quantity['unit']:<10}")

    #confirm if the user wants to add the recipe to the cart
    print()
    print(f"Would you like to add {recipe_name} to your cart? (y/n)")
    user_input = input("Enter your choice: ")

    #check if the user input is valid
    if user_input.lower() not in ["y", "n"]:
        print("Sorry, that is not a valid option. Please try again.")
        add_recipe_to_cart(recipe_name)

    #if the user input is "y", add the recipe to the cart
    if user_input.lower() == "y":
        for ingredient, quantity in recipe["ingredients"].items():
            if ingredient not in cart:
                cart[ingredient] = quantity
                cart[ingredient]["type"] = recipe['cuisine']
                cart[ingredient]['category'] = recipe_name
            else:
                cart[ingredient]["quantity"] += quantity["quantity"]

        print(f"{recipe_name} has been added to your cart.")
        chat_loop()

    #if the user input is "n", go back to the main menu
    if user_input.lower() == "n":
        chat_loop()


In [27]:
#5 View & Edit Cart

def view_cart(optional = None):
    
    #heler
    def get_user_choice(user_input, options_dict):
        try:
            # check if user input is a number
            if user_input.isdigit():
                user_input = int(user_input)
                if user_input in range(1, len(options_dict)+1):
                    return list(options_dict.keys())[user_input-1]

            # check if user input matches an option
            for option in options_dict:
                if user_input.lower() in option.lower() or user_input.lower() in options_dict[option]['title'].lower():
                    return option


            return None
        except:
            print("invalid input, please try again!")
            return None

    def print_options(items_dict):
        # print keys with numbers in consecutive order and add call name attribute f"{item_num}. {item}"
        item_num = 1
        for item in items_dict:
            print(f"{item_num}. {item}")
            item_num += 1


    # function to print the cart in pretty format

    def print_cart(cart_dict):
        """
        Prints the cart dictionary in a well-formatted ASCII table with item, quantity, and unit,  total price & grand total.

        Parameters:
        cart_dict (dict): The dictionary of items in the cart.
        """
        # print the cart table header
        print("*" * 75)
        print("Your cart:")
        print("-" * 75)
        print(f"{'#':<5}{'Item':<25}{'Quantity':<10}{'Unit':<10}{'Price':<10}{'Total Price':<10}")
        print("-" * 75)

        # print the cart table rows
        grand_total = 0
        for i, item in enumerate(cart_dict, start=1):
            #total price based on quantity from cart and unit price from ingredients dictionary
            total_price = cart_dict[item]['quantity'] * grocery_ingredients[item]['price']
            grand_total += total_price
            print(f"{i:<5}{item:<25}{cart_dict[item]['quantity']:<10}{cart_dict[item]['unit']:<10}{grocery_ingredients[item]['price']:<10}${total_price:<10}")

        # print the cart table footer
        print("-" * 75)
        print(f"{'Grand Total':<60}${grand_total:<10}")
        print("*" * 75)

    # function to edit the cart

    def edit_cart(cart_dict):

        # user input to edit cart
        print()
        user_input = input("Enter the item number to edit the quantity or press enter to go back: ")

        # check if user input matches an item in the cart
        item = get_user_choice(user_input, cart_dict)

        # if user input matches an item in the cart, edit the quantity
        if item:
            #print the item and quantity
            print('you currently have', cart_dict[item]['quantity'], cart_dict[item]['unit'], item, 'in your cart')
            print()
            # get the new quantity from the user
            new_quantity = input("Enter the new quantity or press enter to go back to main menu: ")

            # if the user enters a new quantity, update the quantity in the cart
            if new_quantity:
                cart_dict[item]['quantity'] = int(new_quantity)
                print()
                print("Cart updated!")
                print_cart(cart_dict)
                edit_cart(cart_dict)

    # Print welcome message
    print('-'*50)
    print("Welcome to view your cart!")
    print('-'*50)

    #print the cart
    print_cart(cart)
    print()

    # Dictionory of cart actions
    cart_actions = {
        "Edit Cart": edit_cart,
        "Checkout": view_checkout,
        "Chat": chat_loop,
    }

    # Loop for user to select an action until they are done
    while True:
        # Print the list of actions
        print("Please select an action from the list below:\n")
        print_options(cart_actions)

        # Get user input
        user_input = input("Enter your choice: ")

        # Check if the user input matches an action
        try:
            user_choice = get_user_choice(user_input, cart_actions)
        except:
            print("Sorry, that is not a valid option. Please try again.")
            continue

        # If the user input matches an action, run the action and give arguments if needed
        if user_choice:
            #good choice
            cart_actions[user_choice](cart)


In [28]:
#6 Checkout


def view_checkout(optional = None):
    #heler
    def get_user_choice(user_input, options_dict):
        try:
            # check if user input is a number
            if user_input.isdigit():
                user_input = int(user_input)
                if user_input in range(1, len(options_dict)+1):
                    return list(options_dict.keys())[user_input-1]

            # check if user input matches an option
            for option in options_dict:
                if user_input.lower() in option.lower() or user_input.lower() in options_dict[option]['title'].lower():
                    return option


            return None
        except:
            print("invalid input, please try again!")
            return None

    def print_options(items_dict):
        # print keys with numbers in consecutive order and add call name attribute f"{item_num}. {item}"
        item_num = 1
        for item in items_dict:
            print(f"{item_num}. {item}")
            item_num += 1


    # function to print the cart in pretty format

    def print_cart(cart_dict):
        
        # print the cart table header
        print("*" * 75)
        print("Your cart:")
        print("-" * 75)
        print(f"{'#':<5}{'Item':<25}{'Quantity':<10}{'Unit':<10}{'Price':<10}{'Total Price':<10}")
        print("-" * 75)

        # print the cart table rows
        grand_total = 0
        for i, item in enumerate(cart_dict, start=1):
            #total price based on quantity from cart and unit price from ingredients dictionary
            total_price = cart_dict[item]['quantity'] * grocery_ingredients[item]['price']
            grand_total += total_price
            print(f"{i:<5}{item:<25}{cart_dict[item]['quantity']:<10}{cart_dict[item]['unit']:<10}{grocery_ingredients[item]['price']:<10}${total_price:<10}")

        # print the cart table footer
        print("-" * 75)
        print(f"{'Grand Total':<60}${grand_total:<10}")
        print("*" * 75)

    # function to edit the cart

    def edit_cart(cart_dict):

        #print the cart
        print_cart(cart_dict)

        # user input to edit cart
        user_input = input("Enter the item number to edit the quantity or press enter to go back: ")

        # check if user input matches an item in the cart
        item = get_user_choice(user_input, cart_dict)

        # if user input matches an item in the cart, edit the quantity
        if item:
            #print the item and quantity
            print('you currently have', cart_dict[item]['quantity'], cart_dict[item]['unit'], item, 'in your cart')

            # get the new quantity from the user
            new_quantity = input("Enter the new quantity or press enter to go back to main menu: ")

            # if the user enters a new quantity, update the quantity in the cart
            if new_quantity:
                cart_dict[item]['quantity'] = int(new_quantity)
                print("Cart updated!")
                print_cart(cart_dict)
                edit_cart(cart_dict)

    # Print welcome message
    print('-'*50)
    print("Welcome to Checkout! Please check your cart and proceed to payment.")
    print('-'*50)

    #print the cart
    print_cart(cart)
    print()

    # Read customer info from CSV file
    customer_info = {}
    cutomer_order_no = {}
    with open('customer_info.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)
        #check if phone number exists in customer_info.csv
        for row in reader:
            customer_info[row[0]] = row[1]
            cutomer_order_no[row[0]] = row[2]

    #input phone number
    phone = input("Please enter your phone number: ")
    print()
    print("-" * 50)


    #check if phone number exists in customer_info dictionary
    if phone in customer_info:
        print(f"This is your {ordinal(int(cutomer_order_no[phone]))} order with us!...getting addicted? :)")
        print()
        print("I got your last address!")
        print()
        print(customer_info[phone])
        print("-" * 50)
        auto_fill = input("Thats where you want this order, eh? (y/n): ")
        if auto_fill == 'y':
            address = customer_info[phone]
        else:
            address = input("Please enter your address: ")
    else:
        address = input("Please enter your address: ")

    #generate order number
    order_no = random.randint(100000, 999999)

    #store cart and customer info in CSV files
    store_checkout_info(cart, address, phone, order_no)

    #print contact info
    print("Your contact info:")
    print("Address:", address)
    print("Phone:", phone)

    grand_total = 0
    for item in cart:
        total_price = cart[item]['quantity'] * grocery_ingredients[item]['price']
        grand_total += total_price

    #confirm checkout
    confirm = input("Please confirm your order by typing 'yes' or 'no': ")
    if confirm == 'yes':
        print('-'*50)
        print(f'Your total is ${grand_total}')
        print("Yo! Don' worry about the money, we gucci!")
        print()
        print(f"Your order number is {order_no} and will be delivered to {address} within 1 hour.")
        print("Thank you for your order! ")
        print("\n\n\n Dont forget to tip your delivery person!")
        print('-'*50)
        #print large heart

        print("  ___   ___  ")
        print(" /   \\ /   \\ ")
        print("|     |     |")
        print("|           |")
        print(" \\         / ")
        print("   \\     /   ")
        print("     \\ /     ")
        print("      V     ")
        print("  peace & love")

        #exit program
        exit()
        

    elif confirm == 'no':
        chat_loop()


In [29]:
#misc functions

############# Show me

def show_recipes(optional = None):
    print_categorized_items(recipes, "cuisine")

def show_ingredients(optional = None):
    print_categorized_items(grocery_ingredients, "category")

def random_cases():
    print("Yooooo! let me help you out!")
    print("here are some things you can do:")
    
    print("search products - type milk, eggs, etc.")
    print("search by product type category - fruits, produce, etc.")
    print("search recipes - type tacos, soup, etc.")
    print("search by cuisine - type mexican, american, etc.")
    print("view cart - type cart, view cart, etc.")
    print("checkout - type checkout, pay, etc.")


In [30]:
#7 Sales Analytics (keyword: mastergogo)


import matplotlib.pyplot as plt

def analyze_sales(optional = None):
    # Read the CSV data
    data = pd.read_csv('orders.csv')

    # Group by item and aggregate the order count and total value
    item_summary = data.groupby('item').agg({'total qty': 'count', 'total price': 'sum'}).reset_index()
    item_summary.columns = ['item', 'order_count', 'total_value']

    # Plot top 10 items by order count
    item_summary.sort_values('order_count', ascending=False, inplace=True)
    top_10_items_by_order_count = item_summary.head(10)

    # Filter out the data if 'XX' in type
    data = data[data['type'] != 'XX']

    # Group by category and aggregate unique order count and total value
    category_summary = data.groupby('cat').agg({'order no.': 'nunique', 'total price': 'sum'}).reset_index()
    category_summary.columns = ['cat', 'unique_order_count', 'total_value']

    # Group by type and aggregate unique order count and total value
    type_summary = data.groupby('type').agg({'order no.': 'nunique', 'total price': 'sum'}).reset_index()
    type_summary.columns = ['type', 'unique_order_count', 'total_value']

    # Plot top 10 categories by unique order count
    category_summary.sort_values('unique_order_count', ascending=False, inplace=True)
    top_10_categories_by_order_count = category_summary.head(10)

    # Plot top 10 types by unique order count
    type_summary.sort_values('unique_order_count', ascending=False, inplace=True)
    top_10_types_by_order_count = type_summary.head(10)

    # Dual-axis plots
    def dual_axis_plot(data, x_col, y1_col, y2_col, xlabel, title):
        fig, ax1 = plt.subplots(figsize=(10, 6))
        ax2 = ax1.twinx()

        # Sort data by total value (y2_col)
        data = data.sort_values(by=y2_col, ascending=False)

        width = 0.4
        x = list(range(len(data[x_col])))  # Use Python's built-in range function
        x_labels = data[x_col]

        ax1.bar([i - width/2 for i in x], data[y1_col], width=width, color='tab:blue', alpha=0.6, label='Order Count')
        ax2.bar([i + width/2 for i in x], data[y2_col], width=width, color='tab:red', alpha=0.6, label='Total Sales ($)')

        ax1.set_xlabel(xlabel)
        ax1.set_ylabel('Order Count', color='tab:blue')
        ax2.set_ylabel('Total Sales ($)', color='tab:red')

        ax1.tick_params(axis='y', labelcolor='tab:blue')
        ax2.tick_params(axis='y', labelcolor='tab:red')

        ax1.set_xticks(x)
        ax1.set_xticklabels(x_labels, rotation=45, ha='right')

        plt.title(title)
        fig.legend(loc="upper right", bbox_to_anchor=(1, 1), bbox_transform=ax1.transAxes)
        plt.show()



    print('-'*50)
    print('welcome master. the dude abides!')
    print('here are your sales analytics:')
    print('-'*50)

    # Dual-axis plot for top 10 items
    dual_axis_plot(top_10_items_by_order_count, 'item', 'order_count', 'total_value', 'Item', 'Top Items by Order Count & Total Sales ($)')

    # Dual-axis plot for top 10 categories
    dual_axis_plot(top_10_categories_by_order_count, 'cat', 'unique_order_count', 'total_value', 'Category', 'Top Recipes by Order Count & Total Sales ($)')

    # Dual-axis plot for top 10 types
    dual_axis_plot(top_10_types_by_order_count, 'type', 'unique_order_count', 'total_value', 'Type', 'Top Cusines by Unique Order Count & Total Sales ($)')


Chat Program

In [ ]:
############################### CHAT FUNCTION ###############################

############# ACTION DICTIONARIES

#recipes
recipe_keywords = list(recipes.keys())
cusine_keywords = list({recipes[dish]["cuisine"].lower() for dish in recipes})

#ingredients
ingridient_keywords = list(grocery_ingredients.keys())
ingridient_category_keywords = list({grocery_ingredients[ing]["category"] for ing in grocery_ingredients}) 

#navigation

cart_keywords = ["cart", "shopping cart", "shopping"]
checkout_keywords = ["checkout", "pay", "paying", "payment", "pay now", "pay now please", "pay now please!"]
recipe_list_keywords= ["recipe", "recipes", "dish", "dishes", "food", "foods", "meal", "meals", "cuisine", "cuisines"]
ingridient_list_keywords = ["ingredient", "ingredients", "grocery", "groceries", "food", "foods", "item", "items", 'category', 'categories']
random_keywords = ["what", 'what?', "what's", "what's?", "what is", "what is?", "what's up", "what's up?", "what's up dude", "what's up dude?", "what's up dude!", "help", "help!", "help me", "help me!", "help me please", "help me please!", "help me please!, I'm lost"]
master_key = 'mastergogo'

############# ACTION FUNCTIONS
def random_cases(optional = None):
    print("Yooooo! let me help you out!")
    print("here are some things you can do:")
    print()
    print("search products - type milk, eggs, etc.")
    print("search by product type category - fruits, produce, etc.")
    print("search recipes - type tacos, soup, etc.")
    print("search by cuisine - type mexican, american, etc.")
    print("view cart - type cart, view cart, etc.")
    print("checkout - type checkout, pay, etc.")

#Action diciotnary
action_dict = {
        'found_ingridient': {'function': add_ingredient_to_cart, 'counter': 0, 'keywords': ingridient_keywords},
        'found_ingridient_category': {'function': ingridient_category, 'counter': 0, 'keywords': ingridient_category_keywords},
        'found_recipe': {'function': add_recipe_to_cart, 'counter': 0, 'keywords': recipe_keywords},
        'found_cusine': {'function': recipe_cuisine, 'counter': 0, 'keywords': cusine_keywords},
        'found_recipe_list': {'function': show_recipes, 'counter': 0,'keywords': recipe_list_keywords},
        'found_ingridient_list': {'function': show_ingredients, 'counter': 0, 'keywords': ingridient_list_keywords},
        'found_cart': {'function': view_cart, 'counter': 0, 'keywords': cart_keywords},
        'found_checkout': {'function': view_checkout, 'counter': 0, 'keywords': checkout_keywords},
        'found_random': {'function': random_cases, 'counter': 0, 'keywords': random_keywords},
        'found_master': {'function': analyze_sales, 'counter': 0, 'keywords': [master_key]}
    }


##function to convert list into empty dictionary
def list_to_dict(list):
    dict = {}
    for item in list:
        dict[item] = 0
    return dict

#categorize input and call the correct function using the action dictionary and matc_dict_keys function
def categorize_call(user_input):

    for action in action_dict:
        keyword_dict = list_to_dict(action_dict[action]['keywords'])
        matching_keys = match_master(user_input, keyword_dict)
        #add length of matching keys to counter
        action_dict[action]['counter'] += len(matching_keys)
    
    #find the action with the highest counter
    max_counter = 0
    max_action = None
    for action in action_dict:
        if action_dict[action]['counter'] > max_counter:
            max_counter = action_dict[action]['counter']
            max_action = action_dict[action]['function']

    #reset counters
    for action in action_dict:
        action_dict[action]['counter'] = 0

    return max_action

############# CHAT LOOP
def welcome():
    print()
    print('-'*50)
    print("What's up dude! I am GroceryBro. \n\nI am here you find recipes, ingredients, check out, and more!")
    print()
    print('start by typing a product name, category, recipe, or cuisine')
    print('just type. I got you!')


def chat_loop(optional = None):
    
        #initialize user input
        user_input = ""
    
        #chat loop
        while user_input != "bye":
            print()
            print('-'*50)
            user_input = input("What would you like to do? ")
            print('-'*50)
            print()
            user_input = user_input.lower()
            if user_input == "bye":
                print("Bye! Have a nice day!")
            else:
                action = categorize_call(user_input)
                if action:
                    action(user_input)
                else:
                    random_cases(user_input)

welcome()
chat_loop()


--------------------------------------------------
What's up dude! I am GroceryBro. 

I am here you find recipes, ingredients, check out, and more!

start by typing a product name, category, recipe, or cuisine
just type. I got you!

--------------------------------------------------
What would you like to do? what can i do
--------------------------------------------------

Yooooo! let me help you out!
here are some things you can do:

search products - type milk, eggs, etc.
search by product type category - fruits, produce, etc.
search recipes - type tacos, soup, etc.
search by cuisine - type mexican, american, etc.
view cart - type cart, view cart, etc.
checkout - type checkout, pay, etc.

--------------------------------------------------
What would you like to do? search products
--------------------------------------------------

Yooooo! let me help you out!
here are some things you can do:

search products - type milk, eggs, etc.
search by product type category - fruits, produce